In [9]:
import os 
import cv2
import numpy as np
import sys
import time
import math
from PIL import Image
import gdown
import face_recognition

In [ ]:
drive_video_links = {
    "video1": "https://drive.google.com/uc?id=FILE_ID_1",
    "video2": "https://drive.google.com/uc?id=FILE_ID_2",
    "video3": "https://drive.google.com/uc?id=FILE_ID_3",
}

In [ ]:
import gdown
import os
import face_recognition
import cv2

def download_video(video_url, save_as):
    gdown.download(video_url, save_as, quiet=False)
    return save_as

def load_face_encoding(reference_image_path):
    image = face_recognition.load_image_file(reference_image_path)
    encodings = face_recognition.face_encodings(image)
    return encodings[0] if encodings else None

def extract_matching_frames(video_path, reference_encoding, output_dir, frame_skip=5, tolerance=0.6):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    cap = cv2.VideoCapture(video_path)
    frame_index = 0
    matched_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_index % frame_skip == 0:
            rgb_frame = frame[:, :, ::-1]
            face_locations = face_recognition.face_locations(rgb_frame)
            face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

            for encoding in face_encodings:
                result = face_recognition.compare_faces([reference_encoding], encoding, tolerance)
                if result[0]:
                    match_path = os.path.join(output_dir, f"frame_{frame_index}.jpg")
                    cv2.imwrite(match_path, frame)
                    matched_count += 1
                    break  # Only save first matching face per frame

        frame_index += 1

    cap.release()
    print(f"✅ Processed {video_path}, found {matched_count} matching frames.")

# ------------------------------
# USER INPUT
reference_image = "path/to/reference.jpg"
drive_video_links = {
    "video1": "https://drive.google.com/uc?id=FILE_ID_1",
    "video2": "https://drive.google.com/uc?id=FILE_ID_2",
    "video3": "https://drive.google.com/uc?id=FILE_ID_3"
}
# ------------------------------

reference_encoding = load_face_encoding(reference_image)
if reference_encoding is None:
    print("❌ No face found in the reference image.")
    exit()

for name, link in drive_video_links.items():
    video_file = f"{name}.mp4"
    print(f"\n⬇️ Downloading {name}")
    download_video(link, video_file)

    output_dir = f"matched_frames_{name}"
    extract_matching_frames(video_file, reference_encoding, output_dir)
